This is a test project.
Please use it only for training your model 

In [154]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [157]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import LSTM, GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

## Dataset

The dataset is very small so try to find a better enm models.

### Load Data

In [158]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')


print('Dataset Loaded')

Dataset Loaded


In [159]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn  and it is snowy in april 
small_vocab_fr Line 1:  new jersey est parfois calme pendant l automne  et il est neigeux en avril 
small_vocab_en Line 2:  the united states is usually chilly during july  and it is usually freezing in november 
small_vocab_fr Line 2:  les ã©tatsunis est gã©nã©ralement froid en juillet  et il gã¨le habituellement en novembre 


## Pre-processing

### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [162]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.strip().split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.strip().split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1552503 English words.
199 unique English words.
10 Most common words in the English dataset:
"is" "in" "it" "during" "the" "but" "and" "sometimes" "usually" "never"

1701402 French words.
345 unique French words.
10 Most common words in the French dataset:
"est" "en" "il" "les" "mais" "et" "la" "parfois" "jamais" "le"


## Preprocess
We need to convert our textual data into numbers, so that we can feed it into our neural network model. We will be using the following preprocess methods to do that;
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

### Tokenization
We can turn each character into a number or each word into a number. These are called character and word ids, respectively. Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

We will use Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function to tokenize `english_sentences` and `french_sentences`.

In [163]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    x_tk = Tokenizer()
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding
When batching the sequence of word ids together, each sequence needs to be the same length. Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

All the English sequences and the French sequences should have the same length; in order to do that we will be padding the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [164]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    if length is None:
        length = len(max(x, key=len))

    return pad_sequences(x, maxlen=length, padding='post')



# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline

In [165]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)+1
french_vocab_size = len(french_tokenizer.word_index)+1

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 200
French vocabulary size: 346


## Models

Experimenting with various neural network architectures. I'll be training these four simple architectures:
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is a Encoder-Decoder RNN
- Model 5 is a Bidirectional RNN with Embedding

### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.

In [166]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [172]:
import pickle
with(open("./model/en-fr-token.pickle","wb")) as fl:
    pickle.dump([english_tokenizer,french_tokenizer],fl)
    fl.close()


### Model 1: simple RNN 
A basic RNN model is a good baseline for sequence data.

In [145]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Build the layers
    learning_rate = 1e-3
    
    input_seq = Input(input_shape[1:])
    rnn = GRU(64, return_sequences=True)(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(rnn)
    
    model = Model(inputs=input_seq, outputs=logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27572 samples
Epoch 1/10
  1024/110288 [..............................] - ETA: 5:52:34 - loss: 5.8622 - acc: 0.1281

InvalidArgumentError: Received a label value of 8240 which is outside the valid range of [0, 346).  Label values: 99 97 108 105 102 111 114 110 105 101 32 101 115 116 32 103 227 169 110 227 169 114 97 108 101 109 101 110 116 32 99 104 97 117 100 32 112 101 110 100 97 110 116 32 108 32 97 117 116 111 109 110 101 32 32 101 116 32 105 108 32 101 115 116 32 112 97 114 102 111 105 115 32 102 114 111 105 100 32 101 110 32 111 99 116 111 98 114 101 32 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 108 101 115 32 227 169 116 97 116 115 117 110 105 115 32 101 115 116 32 103 227 169 110 227 169 114 97 108 101 109 101 110 116 32 109 101 114 118 101 105 108 108 101 117 120 32 97 117 32 109 111 105 115 32 100 32 97 111 227 187 116 32 32 109 97 105 115 32 105 108 32 101 115 116 32 103 227 169 110 227 169 114 97 108 101 109 101 110 116 32 97 103 114 227 169 97 98 108 101 32 101 110 32 110 111 118 101 109 98 114 101 32 0 0 0 97 105 109 97 105 116 32 32 105 108 32 227 160 32 112 97 114 105 115 32 0 

### Model 2: RNN with Embedding
Converting the words into ids is a good step to preprocess your input inorder to feed them to neural network model, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, I have built a simple RNN with Embeddings.

In [57]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 1e-3
    
    input_seq = Input(input_shape[1:])
    embed_layer = Embedding(english_vocab_size, 64, input_length=output_sequence_length)(input_seq)
    rnn = Bidirectional(GRU(64, return_sequences=True))(embed_layer)

    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(rnn)

    model = Model(inputs=input_seq, outputs=logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
        
    return model



# Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
# tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
embed_rnn_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
embed_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=800, epochs=30, validation_split=0.2)
import pickle
fl = open("./model/en-fr.pickle","wb")
pickle.dump(embed_rnn_model,fl)
fl.close()
# Print prediction(s)
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27572 samples
Epoch 1/30
110288/110288 [==============================] - 222s 2ms/step - loss: 3.2306 - acc: 0.4443 - val_loss: 2.4192 - val_acc: 0.4964
Epoch 2/30
110288/110288 [==============================] - 206s 2ms/step - loss: 1.9842 - acc: 0.5484 - val_loss: 1.6048 - val_acc: 0.6078
Epoch 3/30
110288/110288 [==============================] - 208s 2ms/step - loss: 1.4011 - acc: 0.6553 - val_loss: 1.2137 - val_acc: 0.6987
Epoch 4/30
110288/110288 [==============================] - 203s 2ms/step - loss: 1.0825 - acc: 0.7249 - val_loss: 0.9653 - val_acc: 0.7499
Epoch 5/30
110288/110288 [==============================] - 210s 2ms/step - loss: 0.8811 - acc: 0.7663 - val_loss: 0.7982 - val_acc: 0.7837
Epoch 6/30
110288/110288 [==============================] - 201s 2ms/step - loss: 0.7331 - acc: 0.7980 - val_loss: 0.6708 - val_acc: 0.8130
Epoch 7/30
110288/110288 [==============================] - 201s 2ms/step - loss: 0.6216 - acc: 0.8251 - val_

In [63]:
modl = open("./model/en-fr.pickle","rb")
kmodl = pickle.load(modl)
modl.close()

In [177]:
import string
x_tk = english_tokenizer
x = preproc_english_sentences
y = preproc_french_sentences
y_tk = french_tokenizer
y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
y_id_to_word[0] = ''
sentence = 'i car you'

sentence = [x_tk.word_index[word] for word in sentence.split()]
sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
x = pad(x,21)
sentence = pad(sentence,21)
sentences = np.array([sentence[0], x[0]])
print(sentence)
predictions = kmodl.predict(sentences, len(sentences))
print('Sample 1:')
print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))


[[ 96 102  98   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0]]
Sample 1:
je mars est                  


In [60]:
from keras.models import load_model

### Model 3: Bidirectional RNNs
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in. They are able to see the future data. So, in this model I have experimented with a simple Bidirectional RNN.

In [12]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 1e-3
    
    input_seq = Input(input_shape[1:])
    bdrnn = Bidirectional(GRU(64, return_sequences=True))(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(bdrnn)
    model = Model(inputs=input_seq, outputs=logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_bd_model(bd_model)

# Train and Print prediction(s)
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

bdrnn_model = bd_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
bdrnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

print(logits_to_text(bdrnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 95s 860us/step - loss: 3.2715 - acc: 0.4676 - val_loss: 2.1548 - val_acc: 0.5357
Epoch 2/10
110288/110288 [==============================] - 96s 874us/step - loss: 1.9065 - acc: 0.5651 - val_loss: 1.7098 - val_acc: 0.5862
Epoch 3/10
110288/110288 [==============================] - 96s 872us/step - loss: 1.6051 - acc: 0.5981 - val_loss: 1.5209 - val_acc: 0.6105
Epoch 4/10
110288/110288 [==============================] - 92s 834us/step - loss: 1.4675 - acc: 0.6167 - val_loss: 1.4188 - val_acc: 0.6222
Epoch 5/10
110288/110288 [==============================] - 95s 860us/step - loss: 1.3817 - acc: 0.6296 - val_loss: 1.3464 - val_acc: 0.6336
Epoch 6/10
110288/110288 [==============================] - 96s 871us/step - loss: 1.3182 - acc: 0.6391 - val_loss: 1.2903 - val_acc: 0.6445
Epoch 7/10
110288/110288 [==============================] - 107s 969us/step - loss: 1.2684 - acc: 0.648

### Model 4: Encoder-Decoder RNN
Time to look at encoder-decoder models. This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

In [50]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate = 1e-3
    
    inputs = Input(input_shape[1:])
    encoder = LSTM(64)(inputs)
    decoder = RepeatVector(output_sequence_length)(encoder)
    decoder = LSTM(64, return_sequences=True)(decoder)
    decoded = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(decoder)

    model = Model(inputs=inputs, outputs=decoded)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])

    return model



# Train and Print prediction(s)
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))


encdec_model = encdec_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

encdec_model.fit(tmp_x, preproc_french_sentences, batch_size=1200, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(encdec_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27572 samples
Epoch 1/10
110288/110288 [==============================] - 165s 1ms/step - loss: 3.5972 - acc: 0.4103 - val_loss: 2.8289 - val_acc: 0.4461
Epoch 2/10
110288/110288 [==============================] - 166s 2ms/step - loss: 2.6357 - acc: 0.4624 - val_loss: 2.4966 - val_acc: 0.4738
Epoch 3/10
110288/110288 [==============================] - 169s 2ms/step - loss: 2.3983 - acc: 0.4912 - val_loss: 2.3099 - val_acc: 0.5023
Epoch 4/10
 73200/110288 [==================>...........] - ETA: 52s - loss: 2.2685 - acc: 0.5027

KeyboardInterrupt: 

### Model 5: Bidirectional RNN with Embedding

In [51]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    learning_rate=1e-3

    input_seq = Input(input_shape[1:])
    emb = Embedding(english_vocab_size, 64, input_length=output_sequence_length)(input_seq)
    bdrnn = Bidirectional(GRU(64, return_sequences=True))(emb)
    logits = TimeDistributed(Dense(french_vocab_size, activation='softmax'))(bdrnn)

    model = Model(inputs=input_seq, outputs=logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model


# Pad the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)

# Train
emb_bdrnn = model_final(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

emb_bdrnn.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(emb_bdrnn.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27572 samples
Epoch 1/10
109568/110288 [============================>.] - ETA: 1s - loss: 3.4864 - acc: 0.4269

KeyboardInterrupt: 

## Prediction

In [21]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # Train neural network using Model 5
    max_english_sequence_length = x.shape[1]
    max_french_sequence_length = y.shape[1]
    english_vocab_size = len(x_tk.word_index)+1
    french_vocab_size = len(y_tk.word_index)+1

    # Pad the input
    x = pad(x, max_french_sequence_length)

    # Train
    model = model_final(
        x.shape,
        max_french_sequence_length,
        english_vocab_size,
        french_vocab_size)

    model.fit(x, y, batch_size=1024, epochs=2, validation_split=0.2)
    
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Train on 110288 samples, validate on 27572 samples
Epoch 1/2
110288/110288 [==============================] - 157s 1ms/step - loss: 3.4831 - acc: 0.4255 - val_loss: 2.5109 - val_acc: 0.4779
Epoch 2/2
110288/110288 [==============================] - 149s 1ms/step - loss: 2.2631 - acc: 0.5082 - val_loss: 1.8793 - val_acc: 0.5685
Sample 1:
elle aime aime les les les <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Il a vu un vieux camion jaune
Sample 2:
est est est est est en en et il est est en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
new jersey est parfois calme pendant l automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


In [20]:
x_tk = english_tokenizer
x = preproc_english_sentences
y = preproc_french_sentences
y_tk = french_tokenizer
y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
y_id_to_word[0] = '<PAD>'
sentence = 'he saw a old yellow truck'
sentence = [x_tk.word_index[word] for word in sentence.split()]
sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
sentences = np.array([sentence[0], x[0]])
predictions = model.predict(sentences, len(sentences))
print('Sample 1:')
print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
print('Il a vu un vieux camion jaune')
print('Sample 2:')
print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


NameError: name 'model' is not defined

In [148]:
x_tk = english_tokenizer
x = preproc_english_sentences
y = preproc_french_sentences
y_tk = french_tokenizer
y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
y_id_to_word[0] = '<PAD>'
sentence = 'he saw a old yellow truck'
sentence = [x_tk.word_index[word] for word in sentence.split()]
sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
sentences = np.array([sentence[0], x[0]])
predictions = model.predict(sentences, len(sentences))
print('Sample 1:')
print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
print('Il a vu un vieux camion jaune')

ValueError: Error when checking input: expected input_17 to have shape (21,) but got array with shape (100,)

In [146]:
import pickle
with(open("./model/en-fr.pickle","rb")) as file:
    model = pickle.load(file)